### HIRLAM data, manual download of csv via running of Python script

In [1]:
#https://en.ilmatieteenlaitos.fi/silam-opendata-on-aws-s3
#python get_obs_csv.py --starttime 2010-01-01 --endtime 2010-01-05 --filename obs_2010-01-01_2010_01-05.csv

import pandas as pd
from pathlib import Path

file_name = Path("obs_23_July_2022_to_24_July_2022.csv")
if file_name.exists():
    print("exists, no download required") 
else:
    print("does not exist, download will proceed") 
    !python get_obs_csv.py --starttime 2022-07-23 --endtime 2022-07-24 --filename obs_23_July_2022_to_24_July_2022.csv

exists, no download required


In [2]:
df = pd.read_csv('obs_23_July_2022_to_24_July_2022.csv', sep = ';')

df.head()

,# lat,lon,timestamp,Air temperature (t2m),Wind speed (ws_10min),Gust speed (wg_10min),Wind direction (wd_10min),Relative humidity (rh),Dew-point temperature (td),Precipitation amount (r_1h),Precipitation intensity (ri_10min),Snow depth (snow_aws),Pressure (msl) (p_sea),Horizontal visibility (vis),Cloud amount (n_man),Present weather (auto) (wawa)
0,60.30373,25.54916,1.658534e+09,15.9,1.9,2.8,207.0,95.0,15.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60.30373,25.54916,1.658535e+09,15.9,1.8,3.4,216.0,96.0,15.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60.30373,25.54916,1.658536e+09,16.3,2.0,3.8,266.0,95.0,15.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60.30373,25.54916,1.658536e+09,18.1,3.3,6.0,269.0,92.0,16.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60.30373,25.54916,1.658537e+09,20.0,4.4,6.9,274.0,81.0,16.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### SILAM .ZARR FILES DATA EXTRACTION

In [2]:
!pip install xarray==0.20.2 --quiet  #latest package 2022.6.0 doesn't work

In [3]:
!pip install zarr --quiet

In [4]:
import pandas as pd
import boto3
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import boto3
from botocore.handlers import disable_signing
import os
import zarr

First we download the data from S3. Note that zarr is essentially a directory so we need to download each files (keys) from it.

In [5]:
def download(bucket_name, key, dst_root='/tmp'):
    """ Download zarr directory from S3"""
    resource = boto3.resource('s3')
    resource.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
    
    bucket = resource.Bucket(bucket_name) 
    for object in bucket.objects.filter(Prefix = key):
        dst = dst_root+'/'+object.key
        if not os.path.exists(os.path.dirname(dst)):
            os.makedirs(os.path.dirname(dst))
        resource.Bucket(bucket_name).download_file(object.key, dst)

In [6]:
bucket_name = 'fmi-opendata-silam-surface-zarr'
#key = 'global/20220726/silam_glob_v5_7_1_20220726_CO_d0.zarr'   #26th, CO
#key = 'global/20220707/silam_glob_v5_7_1_20220707_CO_d0.zarr'   #7th, CO
key = 'global/20220707/silam_glob_v5_7_1_20220707_NO_d0.zarr'   #7th, NO

tmp_dir = '/tmp'
tmp_file = tmp_dir + '/'+key

if not os.path.exists(os.path.dirname(tmp_file)):
    os.makedirs(os.path.dirname(tmp_file))

download(bucket_name, key)

Open the data as xarray and print info:

In [7]:
ds = xr.open_zarr(tmp_file)
print(ds.info())

xarray.Dataset {
dimensions:
	time = 24 ;
	lat = 897 ;
	lon = 1800 ;

variables:
	float32 NO(time, lat, lon) ;
		NO:_ChunkSizes = [1, 1, 200, 200] ;
		NO:cell_methods = hybrid: mean ;
		NO:long_name = Concentration in air NO_gas ;
		NO:mode_distribution_type = GAS_PHASE ;
		NO:mode_name =  ;
		NO:mode_solubility = 0 ;
		NO:molar_mass = 0.0300000            kg/mole ;
		NO:number_of_significant_digits = 2 ;
		NO:silam_amount_unit = mole ;
		NO:substance_name = NO ;
		NO:units = ug/m3 ;
	float32 lat(lat) ;
		lat:_ChunkSizes = 200 ;
		lat:_CoordinateAxisType = Lat ;
		lat:axis = Y ;
		lat:long_name = latitude ;
		lat:standard_name = latitude ;
		lat:units = degrees_north ;
	float32 lon(lon) ;
		lon:_ChunkSizes = 200 ;
		lon:_CoordinateAxisType = Lon ;
		lon:axis = X ;
		lon:long_name = longitude ;
		lon:standard_name = longitude ;
		lon:units = degrees_east ;
	datetime64[ns] time(time) ;
		time:_CoordinateAxisType = Time ;
		time:long_name = Forecast time for ForecastModelRunCollection ;
	

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  """Entry point for launching an IPython kernel.


In [8]:
#return size of object in bytes
import sys
sys.getsizeof(ds)

128

#### FILE DEPENDENT

In [9]:
#ds['CO'].sel(time='2022-07-26T18:00:00.000000000').plot(figsize=(20,10))
#ds['CO'].sel(time='2022-07-07T18:00:00.000000000').plot(figsize=(20,10))
ds['NO'].sel(time='2022-07-07T18:00:00.000000000').plot(figsize=(20,10))

In [10]:
ds['NO'].mean(dim='time').plot(figsize=(20,10))

In [11]:
ds['NO'].plot(figsize=(20,10))
plt.title('Histogram')

Text(0.5, 1.0, 'Histogram')

In [12]:
type(ds)

xarray.core.dataset.Dataset

In [13]:
ds

<xarray.Dataset>
Dimensions:  (time: 24, lat: 897, lon: 1800)
Coordinates:
  * lat      (lat) float32 -89.6 -89.4 -89.2 -89.0 -88.8 ... 89.0 89.2 89.4 89.6
  * lon      (lon) float32 -179.8 -179.6 -179.4 -179.2 ... 179.6 179.8 180.0
  * time     (time) datetime64[ns] 2022-07-07T01:00:00 ... 2022-07-08
Data variables:
    NO       (time, lat, lon) float32 dask.array<chunksize=(24, 897, 1800), meta=np.ndarray>
Attributes: (12/16)
    Conventions:               CF-1.4
    NCO:                       "4.5.5"
    SIMULATION_START_DATE:     2022-05-10T00:00:00Z
    cdm_data_type:             GRID
    featureType:               GRID
    geospatial_lat_max:        89.5999984741211
    ...                        ...
    location:                  Proto fmrc:silam_glob_v5_7_1
    nco_openmp_thread_number:  1
    pole_lat:                  -90.0
    pole_lon:                  0.0
    source:                    silam_v5_7 SVN (r596360)
    title:                     SILAM_OUTPUT_ug

In [14]:
df = ds.to_dataframe()

In [15]:
df = df.reset_index()   #flatten down all dimensions

In [16]:
df.head()

,time,lat,lon,NO
0,2022-07-07 01:00:00,-89.599998,-179.800003,5.073275e-12
1,2022-07-07 01:00:00,-89.599998,-179.600006,5.087486e-12
2,2022-07-07 01:00:00,-89.599998,-179.400009,5.073275e-12
3,2022-07-07 01:00:00,-89.599998,-179.199997,5.044853e-12
4,2022-07-07 01:00:00,-89.599998,-179.000000,4.916956e-12


In [17]:
print('Number of NAs:', df.isnull().sum())

Number of NAs: time    0
lat     0
lon     0
NO      0
dtype: int64


In [18]:
import gc

del ds
gc.collect()

122

In [19]:
type(df)

pandas.core.frame.DataFrame

In [20]:
df.head()

,time,lat,lon,NO
0,2022-07-07 01:00:00,-89.599998,-179.800003,5.073275e-12
1,2022-07-07 01:00:00,-89.599998,-179.600006,5.087486e-12
2,2022-07-07 01:00:00,-89.599998,-179.400009,5.073275e-12
3,2022-07-07 01:00:00,-89.599998,-179.199997,5.044853e-12
4,2022-07-07 01:00:00,-89.599998,-179.000000,4.916956e-12


In [21]:
len(df['lat'])

38750400

In [22]:
df['time'].nunique()

24

In [23]:
df.time.value_counts()

2022-07-07 01:00:00    1614600
2022-07-07 02:00:00    1614600
2022-07-07 23:00:00    1614600
2022-07-07 22:00:00    1614600
2022-07-07 21:00:00    1614600
2022-07-07 20:00:00    1614600
2022-07-07 19:00:00    1614600
2022-07-07 18:00:00    1614600
2022-07-07 17:00:00    1614600
2022-07-07 16:00:00    1614600
2022-07-07 15:00:00    1614600
2022-07-07 14:00:00    1614600
2022-07-07 13:00:00    1614600
2022-07-07 12:00:00    1614600
2022-07-07 11:00:00    1614600
2022-07-07 10:00:00    1614600
2022-07-07 09:00:00    1614600
2022-07-07 08:00:00    1614600
2022-07-07 07:00:00    1614600
2022-07-07 06:00:00    1614600
2022-07-07 05:00:00    1614600
2022-07-07 04:00:00    1614600
2022-07-07 03:00:00    1614600
2022-07-08 00:00:00    1614600
Name: time, dtype: int64

In [24]:
#mean CO by time of day
df.groupby(['time'])['NO'].mean()

time
2022-07-07 01:00:00    0.037509
2022-07-07 02:00:00    0.036449
2022-07-07 03:00:00    0.033354
2022-07-07 04:00:00    0.025141
2022-07-07 05:00:00    0.025907
2022-07-07 06:00:00    0.027848
2022-07-07 07:00:00    0.021845
2022-07-07 08:00:00    0.020594
2022-07-07 09:00:00    0.018204
2022-07-07 10:00:00    0.016139
2022-07-07 11:00:00    0.018176
2022-07-07 12:00:00    0.023377
2022-07-07 13:00:00    0.026304
2022-07-07 14:00:00    0.032253
2022-07-07 15:00:00    0.033354
2022-07-07 16:00:00    0.023254
2022-07-07 17:00:00    0.021750
2022-07-07 18:00:00    0.020387
2022-07-07 19:00:00    0.019194
2022-07-07 20:00:00    0.020103
2022-07-07 21:00:00    0.020797
2022-07-07 22:00:00    0.023250
2022-07-07 23:00:00    0.031161
2022-07-08 00:00:00    0.042979
Name: NO, dtype: float32

In [25]:
df['lat'].nunique()

897

In [26]:
df['lon'].nunique()

1800

In [27]:
24*897*1800

38750400

In [28]:
df['NO'].nunique()

10248

In [29]:
df.isnull().sum()

time    0
lat     0
lon     0
NO      0
dtype: int64

In [30]:
df['time'].head()

0   2022-07-07 01:00:00
1   2022-07-07 01:00:00
2   2022-07-07 01:00:00
3   2022-07-07 01:00:00
4   2022-07-07 01:00:00
Name: time, dtype: datetime64[ns]

#### FILE DEPENDENT

In [31]:
#df2 = df[df['time'] == '2022-07-26 12:00:00']   #take slice of data to prevent pandas_profiling from crash, take only midday
df2 = df[df['time'] == '2022-07-07 12:00:00']

In [32]:
lon = df2['lon'].sort_values(ascending = True)
lon = lon.drop_duplicates()
lon = lon[lon > 0]
lon = lon.diff()
lon = lon.dropna()
print('mean difference between longitudinals is: ', lon.mean())

mean difference between longitudinals is:  0.20000002036768283


In [33]:
%%capture
!pip install haversine
#(latitude, longitude)
import haversine as hs
loc1=(-89.599998, 179.800003)
loc2=(-89.599998, 179.600006)

In [34]:
print(hs.haversine(loc1,loc2), 'km')

0.15525472838874982 km


- Currently SILAM supports the following Parameters:
  - PM2.5 - Particulate matter smaller than 2.5 microns
  - PM10 - Particulate matter smaller than 10 microns
  - NO - Nitrogen Monoxide
  - NO₂ - Nitrogen Dioxide
  - SO₂ - Sulfur Dioxide
  - O₃ - Ozone
  - CO - Carbon Monoxide
  - Air density

In [35]:
len(df2)

1614600

In [36]:
#uk data filter
#df2 = df2[(df2['lat'] > 50.10319) & (df2['lat'] < 60.15456)]
#df2 = df2[(df2['lon'] > -7.64133) & (df2['lon'] < 1.75159)]

#london data filter
df2 = df2[(df2['lat'] > 51.239405) & (df2['lat'] < 51.737184)]
df2 = df2[(df2['lon'] > -0.625211) & (df2['lon'] < 0.328289)]

In [37]:
len(df2)

10

In [38]:
!pip install geopandas --quiet
!pip install shapely --quiet

In [39]:
from shapely.geometry import Point, Polygon
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt

In [40]:
lalo_data = df2[['lon', 'lat']]
lalo_data

,lon,lat
19030496,-0.600006,51.400002
19030497,-0.399994,51.400002
19030498,-0.199997,51.400002
19030499,0.000000,51.400002
19030500,0.199997,51.400002
19032296,-0.600006,51.599998
19032297,-0.399994,51.599998
19032298,-0.199997,51.599998
19032299,0.000000,51.599998
19032300,0.199997,51.599998


In [42]:
london_map = gpd.read_file(r'London-wards-2018/London-wards-2018_ESRI/London_Ward_CityMerged.shp')

DriverError: London-wards-2018/London-wards-2018_ESRI/London_Ward_CityMerged.shp: No such file or directory

In [ ]:
geometry = [Point(xy) for xy in zip(df2['lon'], df2['lat'])]
gdf = gpd.GeoDataFrame(lalo_data, geometry = geometry, crs = 4326)
#4326 is the go to CRS for GPS lat lon, need to set here so conversion to another is standardised as setting wrong CRS here means conversion (to_crs) will be wrong 

In [ ]:
london_map['geometry'] = london_map['geometry'].to_crs(epsg=3857)
gdf['geometry'] = gdf['geometry'].to_crs(epsg=3857)

In [ ]:
!pip install contextily --quiet

In [ ]:
import contextily as cx

def makeLayeredMap(*args):
    """This function accepts an arbitrary number of geodataframes, plots them on top of a Contextily basemap. 
    NOTE: Please edit the Plotting-section to specify parameters for the number of layers and the formatting of each layer.
    Output: Saved file and layered map for display."""
    
    # Convert the CRS for all layers to EPSG3857 to match Contextily
    args = list(map(lambda x: x.to_crs(epsg=3857), args))
    # Create figure
    fig, ax = plt.subplots(1, figsize=(20, 20))
    #Set aspect to equal
    ax.set_aspect('equal')
    
    # PLOTTING: Specify layers to plot how to format each layer (colours, transparency, etc.):
    # Layer 1:
    args[0].boundary.plot(ax=ax, color='blue', edgecolor='k', alpha=0.1, zorder=1)
    # Layer 2:
    args[1].plot(ax=ax, color='red', markersize=400, marker='*', zorder=2)
    # ADD LAYERS here as needed:
    #args[2].plot(ax=ax, color='black', alpha=0.3, zorder=3)
    
    # Contextily basemap:
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
    
    # Turn off axis
    ax.axis('off')
    # Save as file
    fig.savefig('silam_london_sensors.png', dpi=300)
    layered_map = plt.show()
    return(layered_map)
    #plt.show()

In [ ]:
makeLayeredMap(london_map, gdf)